# Introduction to Julia
Let's start with a quick overview of the basic syntax.
# Variables and arithmetic
Variables in Julia are created as in Python, Matlab, etc. with a simple assignment operation; variable names can be *arbitrary unicode characters*. Many may be entered in LaTeX notation, using tab substitution: type `\alpha<TAB>`. There is also tab completion on partial names: `\alp<TAB>`

## Title
Even some $\LaTeX$
$$ \int_0^\pi x $$

In [ ]:
# some basic math
x = 1
y = 2

z = x + y

In [ ]:
ans

What about vectors and matrices?

In [ ]:
A = rand(10,300)

In [ ]:
ndims(A)

In [ ]:
A[1, 1] #counting starts at 1!

In [ ]:
typeof(1:5)

In [ ]:
collect(1:5)

It has all of the usual built-in Matlab/Numpy-like linear-algebra and vector functions:

In [ ]:
b = rand(10)
x = A \ b
B = A' * A
erf(eigvals(B)) - 2x.^2 + 4x - 6

Complex numbers and arbitrary-precision arithmetic (via MPFR), of course.

In [ ]:
cos(big(3 + 4im))

## Unicode

All strings are UTF-8 encoded Unicode by default (UTF-16 and UTF-32 also supported):

In [ ]:
matchall(r"\s[a-z]+", "α is a Grëék letter") # regex search of a Unicode string

Like Python 3, variable names can be Unicode, but Julia allows a somewhat wider range of codepoints in identifiers, which can be typed by LaTeX-like tab-completion *\alpha[TAB]\hat[TAB]\_2[TAB]\prime*.

In [ ]:
α̂₂′ = 7
ħ = 6.62606957e-34 / 2π
ẋ = ħ * α̂₂′

Unlike Python 3, Unicode math operators are parsed as infix operators, which are available for user-defined meanings:

In [ ]:
≪(x,y) = x < 0.1*y
50 ≪ 100, 5 ≪ 100, 5 ≤ 50

In [ ]:
const ⊗ = kron
eye(2,2) ⊗ rand(2,2)

## Functions and JIT-compilation

Functions can be defined in several ways, and *don't require type-declarations*.

In [ ]:
# verbose form:
function foo(x)
    return x + 1
end

# one-line form:
foo(x) = x + 1

# anonymous function
x -> x + 1

The *first time you call* a function with arguments of a particular type, Julia JIT-compiles that function *specialized for that type* with LLVM, which is then cached for subsequent calls for the *same types*.

In [ ]:
foo(3) # compiles foo for Int arguments

In [ ]:
foo(7) # re-uses compiled foo(Int)

In [ ]:
foo(7.3) # compiles a different version for Float64 arguments

In [ ]:
foo([1,2,7,9]) # compiles a different version for Array{Int,1} arguments

You can *inspect* the generated code easily at various compilation stages, to see that Julia **uses the type of the arguments to infer the types of expressions** and **can produce C-like fast code**.

In [ ]:
@code_typed foo(3)

In [ ]:
@code_llvm foo(3)

In [ ]:
@code_native foo(3)

In [ ]:
@code_native foo(3.0)

# Example: The Newton method for finding roots of functions 

The Newton method is an iterative method to solve equations of the form $f(x)=0$, i.e. to find *roots* or *zeros* $x^\ast$ such that $f(x^\ast) = 0$. Given an initial guess $x_0$, we repeat the iteration

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}.$$

In [ ]:
f(x) = x^2 - 2

In [ ]:
f(3)

In [ ]:
f′(x) = 2x

Let's implement Newton's method

In [ ]:
x₀ = 3
x = x₀

for i in 1:10
    x_new = x - f(x) / f′(x)
    println(i, "\t", x_new)
    x = x_new
end

with a different x₀ it might find another root

In [ ]:
x₀ = -3
x = x₀

for i in 1:10
    x_new = x - f(x) / f′(x)
    println(i, "\t", x_new)
    x = x_new
end

The Newton method is, in fact, not guaranteed to converge to a root (although it always does so if started "sufficiently close" to a root, at a rate that is known). Furthermore, *which* root it converges to can depend sensitively on the initial condition. Let's calculate this for several initial conditions.

First we create a set of initial conditions on the real line, say between -5 and 5. We now include a step size in the range:

In [ ]:
initial_conditions = -5:0.1:5
collect(initial_conditions)   # use tab completion for long variable names!

For each of these initial conditions, we will run the Newton algorithm for a certain number of steps and store the resulting value. We thus need a new array in which to store the results. One way of creating an array is using the `similar` function, which, by default, creates an array of the same type and same size, but with (currently) uninitialized values:

In [ ]:
roots = similar(initial_conditions)

Now we do the work:

In [ ]:
for (j, x₀) in enumerate(initial_conditions)
    x = x₀

    for i in 1:100
        x = x - f(x) / f′(x)
    end
    
    roots[j] = x
end

In [ ]:
using PyPlot

In [ ]:
plot(initial_conditions, roots)

In [ ]:
@time begin 
    initial_conditions = -100:0.01:100
    roots = similar(initial_conditions)

    for (j, x_0) in enumerate(initial_conditions)
        x = x_0

        for i in 1:1000
            x = x - f(x) / f′(x)
        end

        roots[j] = x
    end
end

If we are used to the performance of C or Fortran, we might start to be unhappy with Julia's speed in this rather simple calculation. A close inspection of the output of the `@time` operation, however, gives us a very important clue: Julia apparently allocated over a gigabyte of memory to do a simple loop with some floating-point numbers! 

This is almost *always* a very strong signal that there is something very wrong in your Julia code! In our case, it is not at all clear what that could be. It turns out to be something very fundamental in Julia: 

[almost] **NEVER WORK WITH GLOBAL OBJECTS!**

Due to technical details about the way that Julia works, it turns out that **GLOBALS ARE BAD**. What is the solution? **PUT EVERYTHING INTO A FUNCTION!** Let's try following this advice. We take *exactly* the same code and just plop it into a new function. For longer functions, Julia has an alternative syntax:

In [ ]:
function do_roots()
    initial_conditions = -100:0.01:100
    roots = similar(initial_conditions)

    for (j, x_0) in enumerate(initial_conditions)
        x = x_0

        for i in 1:1000
            x = x - f(x) / f′(x)
        end

        roots[j] = x
    end
    
    roots
end

In [ ]:
# a semi-colon suppresses output
@time roots = do_roots();
@time roots = do_roots();  

## Complexifying Newton 

The previous result is still pretty boring. It turns out that the Newton method gets interesting if we look for roots of functions of *complex* numbers. [If you are not familiar with complex numbers, you can think of them as pairs of real numbers that have certain mathematical operations defined.]

Let's try to use the Newton method starting from initial conditions distributed in the complex plane, i.e. pairs $a + bi$, where $i = \sqrt{-1}$. First of all let's see how Julia handles complex numbers:

In [ ]:
sqrt(-1)

Oh dear, that didn't work very well. It turns out that Julia is carefully designed to respect, when possible, the type of the input argument. Indeed, let's ask Julia what it thinks `sqrt` means:

We see that `sqrt` is a generic function, with the following methods:

In [ ]:
methods(sqrt)

Julia gives us a list of the available methods, together with links direct to the source code on GitHub (in IJulia) or locally (in Juno).

`sqrt()` acting on a `Float64` returns a `Float64` when it can, or throws a `DomainError` when its argument is negative. To get square roots in the complex plane, we must *start* with a complex number.

## Initial conditions: matrices

We are now ready to think about how to generate a grid of initial conditions of the form $a+bi$ in the complex plane, $\mathbb{C}$. Firstly, we could just iterate over the initial conditions in two repeated `for`s, e.g.

In [ ]:
zeros(3, 3)

In [ ]:
for i in -1:1
    for j in -1:1
        println("($i, $j)")
    end
end

using Array comprehension

In [ ]:
squares = [x^2 for x in 1:6]

Let's do a Newton's method in the complex plane

In [ ]:
f(z) = z^3 - 1
f′(z) = 3z^2

In [ ]:
function newton(x₀, N=100)
    x = x₀
    
    for i in 1:N
        x = x - f(x) / f′(x)
    end
    
    x
end

In [ ]:
function do_complex_roots(range=-5:0.1:5)  # default value
    [newton(b + a*im) for a in range, b in range]
end

In [ ]:
@time roots = do_complex_roots(-3:0.01:3)

In [ ]:
imshow(imag(roots), extent=(-3, 3, -3, 3), cmap="Set3")
text(1, 0, L"1")
text(reim(exp(2π*im/3))..., L"e^{2\frac{\pi}{3}}")
text(reim(exp(-2π*im/3))..., L"e^{-2\frac{\pi}{3}}")

[back to start](0.%20Invitation%20to%20Julia.ipynb)